In [3]:
def process(text:str):
    from parse import compile
    from string import punctuation
    p = compile("{hit:d}.")
    in_list = text.split()
    out_list = list()
    for seg in in_list:
        parse_result = p.parse(seg)
        if parse_result:
            # We got a number with a dot afterward:
            out_list.append(seg.lower())
        else:
            out_list.append(seg.translate(str.maketrans('', '', punctuation)).lower())
    return " ".join(out_list)
import os
def correct_path(s):
    basename = os.path.basename(s)
    current_dir = "/home/peterr/macocu/task8/voxpopuli"
    return os.path.join(current_dir, basename)
import pandas as pd
test = pd.read_csv("vox_populi_test.csv")
dev = pd.read_csv("vox_populi_dev.csv")

for df in [dev, test]:
    df["path"] = df.wavpath.apply(correct_path)
    df["raw_text"] = df.raw_text.apply(process)

In [4]:
test.head(2)

,id,raw_text,normalized_text,speaker_id,split,gender,is_gold_transcript,accent,filename,raw_has_diacritics,path,duration,wavpath
0,20180612-0900-PLENARY-13-hr_20180612-16:32:33_0,gospodine predsjedavajući gospođo povjerenice ...,gospodine predsjedavajući gospođo povjerenice ...,124749,test,female,False,None,20180612-0900-PLENARY-13-hr_20180612-16:32:33_...,True,/home/peterr/macocu/task8/voxpopuli/20180612-0...,25.432,/home/rupnik/macocu/task8/voxpopuli/hr/2018/20...
1,20180612-0900-PLENARY-13-hr_20180612-16:32:33_1,u iranu postoje unutarnji prioriteti na kojima...,u iranu postoje unutarnji prioriteti na kojima...,124749,test,female,False,None,20180612-0900-PLENARY-13-hr_20180612-16:32:33_...,True,/home/peterr/macocu/task8/voxpopuli/20180612-0...,9.784,/home/rupnik/macocu/task8/voxpopuli/hr/2018/20...


In [6]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

import pandas as pd
import os
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import soundfile as sf
import os
# Use old or new vocab? 
os.system("cp vocab_300.json vocab.json")



tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
        "/home/peterr/macocu/task8/", unk_token="[UNK]", 
        pad_token="[PAD]", word_delimiter_token=" ")
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained("/home/peterr/macocu/task8/25_normalised_200h/checkpoint-2720").cuda()

def transcribe(path):
    speech, sample_rate = sf.read(path)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription

for df in [test, dev]:
    df["predicted"] = df.path.apply(transcribe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
for name, df in zip(["dev", "test"], [dev, test]):
    cer = cer_metric.compute(
        references = df.raw_text,
        predictions = df.predicted
    )
    wer = wer_metric.compute(
        references = df.raw_text,
        predictions = df.predicted
    )

    print(f"split: {name}, {cer=:0.4f}, {wer=:0.4f}")

split: dev, cer=0.0670, wer=0.1445
split: test, cer=0.0743, wer=0.1470


In [8]:
dev.to_csv("28_eval_results_dev.csv", index=False)
test.to_csv("28_eval_results_test.csv", index=False)

In [11]:
transcribe("sample_mono.wav")

'proglašava druga i oseba broa fita za predsjednika republike bez ograničenja trajanja mandata'